In [2]:
# Подготовка по хакатону

In [4]:
! pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 8.2 MB 28.8 MB/s 


In [5]:
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install -U gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.7 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [7]:
! pip install thread

ERROR: Could not find a version that satisfies the requirement thread (from versions: none)
ERROR: No matching distribution found for thread


In [8]:
import pandas as pd
import pymorphy2
import nltk
import re

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import FreqDist
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import datetime
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm
from gensim.models import *
from gensim import corpora
from threading import Thread
from time import sleep

from google.colab import drive

ModuleNotFoundError: No module named 'google.colab'

In [9]:
dir_hackathon = '/content/gdrive/MyDrive/Colab Notebooks/Hackathon/'
dir_data_in = f'{dir_hackathon}/Data/In'
dir_data_out = f'{dir_hackathon}/Data/Out'

column_num = 'Номер'
column_name = 'Общее наименование продукции'
column_group = 'Группа продукции'

In [10]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
all_data_in = pd.read_excel(f'{dir_data_in}/dataset.xlsx', header=0)
all_data_in.columns = ['Номер продукции', 'Коды ТН ВЭД ЕАЭС', 'Технические регламенты', 'Группа продукции', 'Общее наименование продукции', 
                       'ИЛ', 'Заявитель', 'Адрес Заявителя', 'Изготовитель', 'Страна', 'Адрес изготовителя']
all_data_in.head(2)

,Номер продукции,Коды ТН ВЭД ЕАЭС,Технические регламенты,Группа продукции,Общее наименование продукции,ИЛ,Заявитель,Адрес Заявителя,Изготовитель,Страна,Адрес изготовителя
0,37632 Продукция,4016999708,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Узлы пересечения противопожарных преград кабел...,Проходки кабельные модульные универсальные огн...,ИЛ Мопалия Миддендорфа,ООО Европейский гологлаз 6,РОССИЯ 461220 БАРАБАНОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Ара Леара 11,РОССИЯ,РОССИЯ 182375 ЛУКИНО ПСКОВСКАЯ ОБЛАСТЬ
1,37565 Продукция,7308909809,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,"Противопожарные окна, двери, двери шахт лифтов...",Ворота противопожарные металлические распашные...,ИЛ Мопалия Миддендорфа,ООО Дроздовидная6,РОССИЯ 461182 ШЕСТАКОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Аннамский палочник 11,РОССИЯ,РОССИЯ 182306 ПОДДУБЬЕ ПСКОВСКАЯ ОБЛАСТЬ


In [12]:
# data_name_group = all_data_in[['Группа продукции', 'Общее наименование продукции']]
data_name_group = all_data_in[[column_name]]
data_name_group

,Общее наименование продукции
0,Проходки кабельные модульные универсальные огн...
1,Ворота противопожарные металлические распашные...
2,Клапаны противопожарные марки КПВ.01(EI60) пря...
3,Вентиляторы крышные радиальные дымоудаления с ...
4,Вентиляторы радиальные дымоудаления типов ВР-8...
...,...
66748,Блок пробоотборный обогреваемый
66749,Система отбора проб природного газа
66750,Фильтры
66751,Элементы трубопроводов


In [13]:
# prepare data:
def prepare_data(df_data):
  prepared_data = df_data
  for row in prepared_data.columns:
      prepared_data[row] = prepared_data[row].str.replace(r'\W', ' ')    
      prepared_data[row] = prepared_data[row].str.replace(r'\d', ' ')    
      prepared_data[row] = prepared_data[row].str.lower()
      prepared_data[row] = prepared_data[row].str.replace(r' +', ' ') 

  return df_data

prepared_data = prepare_data(data_name_group)
prepared_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

,Общее наименование продукции
0,проходки кабельные модульные универсальные огн...
1,ворота противопожарные металлические распашные...
2,клапаны противопожарные марки кпв ei прямоугол...
3,вентиляторы крышные радиальные дымоудаления с ...
4,вентиляторы радиальные дымоудаления типов вр д...


In [14]:
# Посмотреть, можеть добавить колонку с нумерацией

In [15]:
nltk.download('stopwords')
nltk.download('punkt')

stop_words = nltk.corpus.stopwords.words("russian")
stop_words.extend(['https', 'который', 'это', 'также', 'новый', 'являться', 'позволять', 'возможность', 
                   'наш', 'свой', 'ru', 'ещё', 'www', 'ru', 'com', 'пока', 'год'])

def lemmatize_cell(data_lemm, x, y):
    lemma = pymorphy2.MorphAnalyzer()
    cell = data_lemm.loc[y, data_lemm.columns[x]]
    print('Coords: ', x, y, '\n') #, datetime.datetime.now()
    text = ''
    if type(cell) == str and len(cell) > 0:
        text = " ".join([lemma.parse(word)[0].normal_form for word in cell.split(' ')])
        tokens = word_tokenize(text, language="russian")
#         tokens = lemma.parse(word)[0].normal_form 
        for token in tokens:
            if token in stop_words:
                tokens.remove(token)
        text = " ".join(tokens)
        
#     print(text, '\n')
        
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
 prepared_data['Номер'] = range(1, len(prepared_data) + 1)
 prepared_data = prepared_data.reindex(columns=['Номер', 'Группа продукции', 'Общее наименование продукции'])
 prepared_data.tail(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Номер,Группа продукции,Общее наименование продукции
66751,66752,NaN,элементы трубопроводов
66752,66753,NaN,станция газозаправочная автомобильная


In [18]:
%%time
data_lemm = prepared_data

data_lemm_out = pd.DataFrame()
data_lemm_out['Номер'] = data_lemm['Номер']
x_max = data_lemm.shape[1]
# x_max = 20
y_max = data_lemm.shape[0]
data_lemm_out = pd.DataFrame(columns=data_lemm.columns)

for x in range(1, x_max): # Прогон, без номера
    for y in range(y_max):  
        # th = Thread(target=lemmatize_cell(data_lemm, x, y))
        # th.start()

        cell = lemmatize_cell(data_lemm, x, y)
        data_lemm_out.loc[y, data_lemm_out.columns[x]] = cell
        
data_lemm_out

Coords:  1 0 

Coords:  1 1 

Coords:  1 2 

Coords:  1 3 

Coords:  1 4 

Coords:  1 5 

Coords:  1 6 

Coords:  1 7 

Coords:  1 8 

Coords:  1 9 

Coords:  1 10 

Coords:  1 11 

Coords:  1 12 

Coords:  1 13 

Coords:  1 14 

Coords:  1 15 

Coords:  1 16 

Coords:  1 17 

Coords:  1 18 

Coords:  1 19 

Coords:  1 20 

Coords:  1 21 

Coords:  1 22 

Coords:  1 23 

Coords:  1 24 

Coords:  1 25 

Coords:  1 26 

Coords:  1 27 

Coords:  1 28 

Coords:  1 29 

Coords:  1 30 

Coords:  1 31 

Coords:  1 32 

Coords:  1 33 

Coords:  1 34 

Coords:  1 35 

Coords:  1 36 

Coords:  1 37 

Coords:  1 38 

Coords:  1 39 

Coords:  1 40 

Coords:  1 41 

Coords:  1 42 

Coords:  1 43 

Coords:  1 44 

Coords:  1 45 

Coords:  1 46 

Coords:  1 47 

Coords:  1 48 

Coords:  1 49 

Coords:  1 50 

Coords:  1 51 

Coords:  1 52 

Coords:  1 53 

Coords:  1 54 

Coords:  1 55 

Coords:  1 56 

Coords:  1 57 

Coords:  1 58 

Coords:  1 59 

Coords:  1 60 

Coords:  1 61 

Coords:  1 62 

Co

KeyboardInterrupt: ignored

In [ ]:
data_lemm_out.head()

In [ ]:
data_lemm_out.to_excel(f'{dir_data_out}/data_lemm_morphy.xlsx', header=0)

In [19]:
data_lemm_out = pd.read_excel(f'{dir_data_out}/data_lemm_morphy.xlsx', header=0)
data_lemm_out.columns = [column_num, column_name, column_group]

data_lemm_out.tail(2)

ValueError: ignored

In [ ]:
data_lemm_out_small = data_lemm_out[:500]
data_lemm_out_small.tail(2)

In [33]:
# Get dataframe
column_code_tn = 'Коды ТН ВЭД ЕАЭС' # !
column_order = 'Технические регламенты' # !
data_lemm_out = pd.read_excel(f'{dir_data_out}/data_lemm_morphy.xlsx', header=0)
data_lemm_out.columns = [column_num, column_name, column_group, column_code_tn, column_order] # Проверить при получении файла, в каком порядке

column_id_group = 'Группа продукции_ordinal'

data_new = pd.read_excel(f'{dir_data_in}/dateset_categorical.xlsx', header=0)
data_claster_out_name = data_lemm_out
data_claster_out_gr = data_new[[column_num, column_id_group]]
data_claster_out = pd.merge(data_claster_out_name, data_claster_out_gr, on="Номер")
data_claster_out.tail(3)

,Номер,Общее наименование продукции,Группа продукции,Коды ТН ВЭД ЕАЭС,Технические регламенты,Группа продукции_ordinal
66749,66750,электрический электрооборудование неэлектричес...,фильтр,Балдосики рулят,Играем по крупному,1939.0
66750,66751,элемент оборудование сборочный единица комплек...,элемент трубопровод,Балдосики рулят,Играем по крупному,2023.0
66751,66752,электрический электрооборудование неэлектричес...,станция газозаправочный автомобильный,Балдосики рулят,Играем по крупному,1939.0


In [61]:
def get_df_top_terms(df, checked_column, top_n=1000):
  common_projects_words = get_top_terms(df, checked_column, top_n)

  data_common_words = pd.DataFrame(list(common_projects_words))
  data_common_words.columns = ['Слово', 'Количество ']

  return data_common_words

def get_top_terms(df, checked_column, top_n=1000):

  # Get list of words
  df[checked_column] = df[checked_column].astype('str')
  words = " ".join(df[checked_column]).split()

  # Get common words in df
  df[checked_column] = df[checked_column].astype('str') # New (for group)
  words = " ".join(df[checked_column]).split()
  common_projects_words = Counter(words).most_common(top_n)

  return common_projects_words

data_common_words = get_df_top_terms(data_claster_out, column_group, 1000)
data_common_words.to_excel(f'{dir_data_out}/data_common_words.xlsx')
data_common_words

,Слово,Количество
0,тот,15305
1,число,14854
2,изделие,14501
3,пряжа,12489
4,нить,11483
...,...,...
995,майк,95
996,прицепной,95
997,корпус,95
998,нанесение,95


In [63]:
def get_df_claster_top_terms(df, checked_column, top_n=50):

  clasters_common_words = df.groupby(column_id_group).apply(lambda x: get_claster_top_terms(x, checked_column, top_n))
  count_clasters = len(clasters_common_words)

  words_info, count_words = map(list, zip(*clasters_common_words)) 
  frequent_words_count = 'Частые слова и их количество'
  total_count_words = 'Общее количество слов'
  data_clasters_common_words = pd.DataFrame(index=range(count_clasters), columns=[frequent_words_count, total_count_words] )
  data_clasters_common_words[frequent_words_count] = words_info
  data_clasters_common_words[total_count_words] = count_words

  pd_clasters_common_words = pd.DataFrame(data_clasters_common_words)

  return pd_clasters_common_words, count_clasters, frequent_words_count, total_count_words # !

# Get top words from clasters
def get_claster_top_terms(df, checked_column, top_n):
    df[checked_column] = df[checked_column].apply(lambda x: str(x)) 
    words = ' '.join(df[checked_column]).split()
    count_words = len(words)  
    list_top_words_and_count = Counter(words).most_common(top_n)

    return list_top_words_and_count, count_words


data_clasters_common_words, count_clasters, frequent_words_count, total_count_words = get_df_claster_top_terms(data_claster_out, column_group, 100)
data_clasters_common_words.to_excel(f'{dir_data_out}/clasters_common_words_gr.xlsx')
data_clasters_common_words

,Частые слова и их количество,Общее количество слов
0,"[(муфта, 2), (трубопроводный, 2), (разъёмный, ...",20
1,"[(управление, 19), (электрический, 13), (аппар...",145
2,"[(comfort, 4), (управление, 2), (air, 2), (апп...",20
3,"[(автопогрузчик, 3), (погрузчик, 2), (машина, ...",13
4,"[(компонент, 1), (поставлять, 1), (качество, 1...",11
...,...,...
2061,"[(изделие, 5), (имитационный, 5), (пиротехниче...",23
2062,"[(бурение, 7), (скважина, 6), (оборудование, 5...",59
2063,"[(станок, 1), (бурение, 1), (скважина, 1), (го...",5
2064,"[(установка, 3), (буровой, 1), (подземный, 1),...",16


In [69]:
def get_metrics(product_name, data_clasters_common_words):
  list_name_words = pd.unique(product_name.split(' '))

  list_k = []
  for id_claster in range(count_clasters): # Внутри кластера суммировать или умножать - подумать    
    # print(range(count_clasters))

    claster_len = data_clasters_common_words[:id_claster][total_count_words] # !!!
    # Tfidf - поискать наименование метрики - ??? 

    # print(f'Claster id: {str(id_claster)}')
    k = 0
    for word in list_name_words:  
      word_name_count = np.sum(list_name_words == word) # !!!
      name_len = len(product_name) # !!! # Необязателен, так, как не изменяется для одной и той же фразы

      claster_words, claster_count_words = map(list,zip(*list(*data_clasters_common_words.loc[1:1, frequent_words_count]))) # Неудачные названия: claster_count_words, word_claster_count
      if word in claster_words:
        id_word = claster_words.index(element) 
        word_claster_count = claster_count_words[id_word] # !!!

        k +=  word_claster_count / (word_name_count * claster_len) # !!!!! Подумать, как правильно !!!

    list_k.append(k)

  return list_k


list_k = get_metrics(product_name, data_clasters_common_words)
list_k

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


''

In [83]:
required_name = get_required_value(product_name, data_claster_out, column_name)
required_name

''

In [84]:
required_code_tn = get_required_value(product_name, data_claster_out, column_code_tn)
required_name

''

In [85]:
required_order = get_required_value(product_name, data_claster_out, column_order)
required_order

''